In [1]:
import pandas as pd
import datetime
data_path = './data/'

In [2]:
timeFrom = datetime.datetime.strptime('2017-01-01 14:00:00', "%Y-%m-%d %H:%M:%S")
timeTo = datetime.datetime.strptime('2018-04-30 23:00:00', "%Y-%m-%d %H:%M:%S")
Hours_Delta = pd.date_range(timeFrom, timeTo, freq='H').strftime('%Y-%m-%d %H:%M:%S')

In [13]:
time_info = pd.DataFrame(Hours_Delta, columns=['time'])

In [14]:
time_info['time_dt'] = pd.to_datetime(time_info['time'])

**For weekday, 0 means Mon, and 6 means Sat.**

In [22]:
time_info['year'] = time_info.apply(lambda row: row[1].year, axis=1)
time_info['month'] = time_info.apply(lambda row: row[1].month, axis=1)
time_info['day'] = time_info.apply(lambda row: row[1].day, axis=1)
time_info['weekday'] = time_info.apply(lambda row: row[1].weekday(), axis=1)
time_info['hour'] = time_info.apply(lambda row: row[1].hour, axis=1)

In [24]:
time_info.to_csv(data_path+'time_info.csv')

**Bulid features about holidays, workdays, etc.**